# Week Summary - MLFlow

# Problem

When testing multiple models, things can get messy
* Rewriting code cells for testing different models
* Keeping track of best parameters through a spreadsheet

In [1]:
import pickle

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris, load_diabetes
from sklearn.metrics import accuracy_score, mean_squared_error, make_scorer

from sklearn.model_selection import KFold, cross_val_score, train_test_split

In [2]:
X, y = load_iris(return_X_y=True)

# 1st test
lr = LogisticRegression(C=0.1, random_state=42).fit(X, y)
y_pred = lr.predict(X)
print(f'Accuracy Logistic Regression: {accuracy_score(y, y_pred)}')

# 2st test
dt = DecisionTreeClassifier(min_samples_split=4, random_state=42).fit(X, y)
y_pred = dt.predict(X)
print(f'Accuracy Decision Tree: {accuracy_score(y, y_pred)}')

# 3 test
# variable overwritten
dt = DecisionTreeClassifier(min_samples_split=2, max_depth=5, random_state=42).fit(X, y)
y_pred = dt.predict(X)
print(f'Accuracy Decision Tree: {accuracy_score(y, y_pred)}')


Accuracy Logistic Regression: 0.96
Accuracy Decision Tree: 0.9866666666666667
Accuracy Decision Tree: 1.0


We want a better to keep track of the experiments tried ang globally check all the combinations and their effects in the final performance.

**Solution** - track experiments

The code below will start a MLFlow experiment locally

In [3]:
import mlflow 

mlflow.set_experiment('experiment_1')

2024/05/27 11:32:49 INFO mlflow.tracking.fluent: Experiment with name 'experiment_1' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///Users/caiomiyashiro/repo/Personal/MLOpsZoomcamp/02-experiment-tracking/mlruns/239494832745782215', creation_time=1716777169074, experiment_id='239494832745782215', last_update_time=1716777169074, lifecycle_stage='active', name='experiment_1', tags={}>

In order to see the results in an UI, we can start a MLFlow UI page

``` bash
pip install mlflow
mlflow ui --backend-store-uri sqlite:///mlflow.db
``` 

You will be able to access the MLFlow UX at http://127.0.0.1:5000/

## First example with simple logging

In [5]:
X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

with mlflow.start_run():
    mlflow.set_tag('developer', 'caio')

    alpha = 0.1
    lr = LogisticRegression(C=alpha, random_state=42)
    mlflow.log_param('alpha', alpha)

    kf = KFold(n_splits=3, shuffle=True, random_state=1)
    rmse_scorer = make_scorer(mean_squared_error)
    cv_results = cross_val_score(lr, X_train, y_train, cv=kf, scoring=rmse_scorer)
    print("Average cross-validation RMSE:", cv_results.mean())
    mlflow.log_metric('rmse',cv_results.mean())
    
    lr.fit(X_train, y_train)

    # y_test_predicted = lr.predict(y_test)
    # test_rmse = mean_squared_error(y_test, y_test_predicted)

    with open('models/lasso.bin', 'wb') as f_out:
        pickle.dump(lr, f_out)
    mlflow.log_artifact(local_path="models/lasso.bin", artifact_path="models_pickle")

Average cross-validation RMSE: 7476.9177832749265


## More complex example with grid tuning and hyperopt

Hyperopt is a faster way to search hiperparameter space (compared to grid search)

After running them, select all runs at mlflow and click "compare"

In [11]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment('experiment_hypteropt_fine_tuning')

2024/05/27 12:10:53 INFO mlflow.tracking.fluent: Experiment with name 'experiment_hypteropt_fine_tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='/Users/caiomiyashiro/repo/Personal/MLOpsZoomcamp/02-experiment-tracking/mlruns/1', creation_time=1716779453213, experiment_id='1', last_update_time=1716779453213, lifecycle_stage='active', name='experiment_hypteropt_fine_tuning', tags={}>

In [7]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_test, label=y_test)

In [13]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=500,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        mlflow.xgboost.log_model(booster, 'models_mlflow')

    return {'loss': rmse, 'status': STATUS_OK}

In [14]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:squarederror',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=20,
    trials=Trials()
)

[0]	validation-rmse:61.94635                          
[1]	validation-rmse:62.33228                          
[2]	validation-rmse:63.52594                          
[3]	validation-rmse:65.74194                          
[4]	validation-rmse:67.89354                          
[5]	validation-rmse:69.66619                          
[6]	validation-rmse:69.29870                          
[7]	validation-rmse:68.93198                          
[8]	validation-rmse:71.02177                          
[9]	validation-rmse:70.65451                          
[10]	validation-rmse:71.21747                         
[11]	validation-rmse:71.91970                         
[12]	validation-rmse:71.39119                         
[13]	validation-rmse:71.26847                         
[14]	validation-rmse:71.91082                         
[15]	validation-rmse:71.92807                         
[16]	validation-rmse:71.76494                         
[17]	validation-rmse:71.96724                         
[18]	valid

/Users/caiomiyashiro/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/Users/caiomiyashiro/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [12:14:50] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:72.46318                                                   
[1]	validation-rmse:70.32559                                                   
[2]	validation-rmse:68.83513                                                   
[3]	validation-rmse:68.32753                                                   
[4]	validation-rmse:68.49121                                                   
[5]	validation-rmse:68.74229                                                   
[6]	validation-rmse:69.25266                                                   
[7]	validation-rmse:69.86757                                                   
[8]	validation-rmse:70.59612                                                   
[9]	validation-rmse:71.24899                                                   
[10]	validation-rmse:71.88880                                                  
[11]	validation-rmse:72.48810                                                  
[12]	validation-rmse:73.07356           

/Users/caiomiyashiro/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/Users/caiomiyashiro/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [12:14:53] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:72.41153                                                   
[1]	validation-rmse:70.05777                                                   
[2]	validation-rmse:68.02974                                                   
[3]	validation-rmse:66.63083                                                   
[4]	validation-rmse:66.03276                                                   
[5]	validation-rmse:64.93220                                                   
[6]	validation-rmse:64.14491                                                   
[7]	validation-rmse:63.67693                                                   
[8]	validation-rmse:63.45522                                                   
[9]	validation-rmse:63.14091                                                   
[10]	validation-rmse:63.13552                                                  
[11]	validation-rmse:62.98417                                                  
[12]	validation-rmse:62.77058           

...

/Users/caiomiyashiro/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(

/Users/caiomiyashiro/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")



[45]	validation-rmse:59.69792
[46]	validation-rmse:59.85109                                                   
[47]	validation-rmse:59.82554                                                   
[48]	validation-rmse:60.00443                                                   
[49]	validation-rmse:60.08613                                                   
[50]	validation-rmse:60.05646                                                   
[51]	validation-rmse:59.87519                                                   
[52]	validation-rmse:59.90100                                                   
[53]	validation-rmse:60.07956                                                   
[54]	validation-rmse:60.01011                                                   
[55]	validation-rmse:60.03202                                                   
[56]	validation-rmse:59.98893                                                   
[57]	validation-rmse:59.96050                                                  

/Users/caiomiyashiro/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/Users/caiomiyashiro/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [12:15:27] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)



100%|██████████| 20/20 [00:38<00:00,  1.92s/trial, best loss: 58.50377587116799]


/Users/caiomiyashiro/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(

/Users/caiomiyashiro/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")



In [31]:
params = {
    'learning_rate': 0.09585355369315604,
    'max_depth': 30,
    'min_child_weight': 1.060597050922164,
    'objective': 'reg:linear',
    'reg_alpha': 0.018060244040060163,
    'reg_lambda': 0.011658731377413597,
    'seed': 42
}

with mlflow.start_run():
    mlflow.set_tag("model", "xgboost")
    mlflow.log_params(params)
    booster = xgb.train(
        params=params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )
    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)
    mlflow.xgboost.log_model(booster, 'models_mlflow')

/Users/caiomiyashiro/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [15:03:23] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:34.43285
[1]	validation-rmse:34.17788
[2]	validation-rmse:33.99347
[3]	validation-rmse:33.86326
[4]	validation-rmse:33.77491
[5]	validation-rmse:33.72211
[6]	validation-rmse:33.69365
[7]	validation-rmse:33.68468
[8]	validation-rmse:33.69479
[9]	validation-rmse:33.71294
[10]	validation-rmse:33.73768
[11]	validation-rmse:33.77522
[12]	validation-rmse:33.81332
[13]	validation-rmse:33.85174
[14]	validation-rmse:33.89416
[15]	validation-rmse:33.94299
[16]	validation-rmse:33.98560
[17]	validation-rmse:34.02745
[18]	validation-rmse:34.06985
[19]	validation-rmse:34.11616
[20]	validation-rmse:34.15489
[21]	validation-rmse:34.19737
[22]	validation-rmse:34.23079
[23]	validation-rmse:34.26960
[24]	validation-rmse:34.29872
[25]	validation-rmse:34.33475
[26]	validation-rmse:34.37394
[27]	validation-rmse:34.40742
[28]	validation-rmse:34.43439
[29]	validation-rmse:34.45618
[30]	validation-rmse:34.47800
[31]	validation-rmse:34.50389
[32]	validation-rmse:34.52565
[33]	validation-rmse

/Users/caiomiyashiro/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Users/caiomiyashiro/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [15:03:54] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


## Loading the model from MLFlow

We check the MLFlow UI and check the run that has achieved the smaller RMSE. We're going to load the model in 2 ways:

1. Loading from a specific run
2. loading from model registry

In [15]:
# "model_mlflow" is the folder defined during model logging
logged_model = 'runs:/fe14c14ff3364b3e89576e1014dd2e1e/models_mlflow' 


# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# OR

xgboost_model = mlflow.xgboost.load_model(logged_model)

# Setting up MLFlow for different scenarios

## Show the different parts of MLFlow
1. Tracking server
2. Backend server
3. Artifact server


## Instatiate remote MLFlow in Azure

1. Start VM. Save public key and public IP address to add to alias. Change key permissions to only be accessible by owner
  
```bash
chmod 600 /Users/caiomiyashiro/.ssh/vm-mlops_key.pem
```
  
2. Start PostgreSQL Database and allow VM's IP to connect
3. Start Storage Container and allow VM's IP to connect

## Connect to VM using bash

1. Writing alias at .ssh/config

``` bash
Host mlops-zoomcamp
   HostName 20.18.227.5
   User azureuser
   IdentityFile ~/.ssh/vm-mlops_key.pem
   StrictHostKeyChecking no
```  

2. Connect to VM using: 
```bash
ssh mlops-zoomcamp
```

3. Install and configure anaconda
```bash
wget https://repo.anaconda.com/archive/Anaconda3-2024.02-1-Linux-x86_64.sh
bash Anaconda3-2024.02-1-Linux-x86_64.sh
# could install docker and docker container

conda create --name mlflow-tracking
conda install jupyter scikit-learn  pandas  seaborn  xgboost azure-storage-blob
pip3 install mlflow psycopg2-binary hyperopt azure-identity
# conda install -c conda-forge psycopg2-binary will update other packages and create a memory error!
# conda install mlflow might create error Unable to display MLflow UI - landing page (index.html) not found.
```

4. Initiate MLFlow with postgresSQL and blobstorage
```bash 
export AZURE_STORAGE_CONNECTION_STRING="<get from storage container>"

mlflow server -h 0.0.0.0 -p 5000 --backend-store-uri postgresql://azureuser:<password>@<hostname>:5432/postgres --default-artifact-root wasbs://<blob container>@<storage account>.blob.core.windows.net

# in case of errors, you can test the database connection from the VM
psql -h mlopspostgress.postgres.database.azure.com -U azureuser -d postgres -p 5432
```


# Locally

1. Install azure client so we can do browser login
```bash
brew update && brew install azure-cli  # for mac or the equivalent for windows
az login                               # login using browser
```

In [24]:
import os # set access to storage container
os.environ["AZURE_STORAGE_CONNECTION_STRING"] = "<get from storage account>"

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_tracking_uri('http://20.18.227.5:5000/')
mlflow.set_experiment("finally")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

/Users/caiomiyashiro/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/Users/caiomiyashiro/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


default artifacts URI: 'wasbs://mlops-blob@mlopsdata37.blob.core.windows.net/1/e732d7eca2c446b2bae55a91ee4237f0/artifacts'
